In [1]:
import torch
import torchvision
import onnx
import onnxruntime as ort
from torchvision import transforms as T
from PIL import Image
import os
from tqdm import tqdm
import time
preprocess_parseq = T.Compose([
            T.Resize((32, 128), T.InterpolationMode.BICUBIC),
            T.ToTensor(),
            T.Normalize(0.5, 0.5)
        ])

In [3]:
batch_size = 4
parseq = torch.hub.load('baudm/parseq', 'parseq', pretrained=True).eval()

output_path = "parseq_{}_b{}_torchscript.pt".format("cpu", batch_size)
ts_model = torch.jit.load(output_path)
img_folder = "/home/ubuntu/parseq/digits_demo"

input_holder = torch.zeros(batch_size, 3, 32, 128)

img_paths = [os.path.join(img_folder, x) for x in os.listdir(img_folder) if x.endswith("jpg")]

N = len(img_paths)
n_batch = N//batch_size + 1

for i in range(n_batch):
    # print(list(range((i-1)*batch_size,batch_size*i)))
    if batch_size*(i+1) < N:
        input_holder = torch.zeros(batch_size, 3, 32, 128)
        img_paths_tmp = img_paths[(i)*batch_size:batch_size*(i+1)]
    else:
        input_holder = torch.zeros(N-(n_batch-1)*batch_size, 3, 32, 128)
        img_paths_tmp = img_paths[(i)*batch_size:N]
    
    print(input_holder.size())
    for j in range(len(img_paths_tmp)):
                
        img_input = Image.open(img_paths_tmp[j]).convert('RGB')
        img_input = preprocess_parseq(img_input.convert('RGB'))
        # Preprocess. Model expects a batch of images with shape: (B, C, H, W)
        
        input_holder[j, :, :, :] = img_input[:,:,:]
        
    with torch.no_grad():  
        start = time.time()
        logits_ts = ts_model(input_holder)
        end_1 = time.time()
        logits = parseq(input_holder)
        end_2 = time.time()

    # print("parseq")
    # print(end_2-end_1)
    # print("torchscript")
    # print(end_1-start)
    pred = logits.softmax(-1)
    label, confidence = parseq.tokenizer.decode(pred)
        
    pred = logits_ts.softmax(-1)
    label_ts, confidence = parseq.tokenizer.decode(pred)
    
    print(label_ts, label)

Using cache found in /home/ubuntu/.cache/torch/hub/baudm_parseq_main


torch.Size([4, 3, 32, 128])


/home/ubuntu/OCR_research/env_ocr_research/lib/python3.7/site-packages/torch/nn/modules/module.py:1130: UserWarning: operator() sees varying value in profiling, ignoring and this should be handled by GUARD logic (Triggered internally at  ../torch/csrc/jit/codegen/cuda/parser.cpp:3513.)
  return forward_call(*input, **kwargs)


['448', '398', '464', '422'] ['448', '398', '464', '422']
torch.Size([4, 3, 32, 128])


/home/ubuntu/OCR_research/env_ocr_research/lib/python3.7/site-packages/torch/nn/modules/module.py:1130: UserWarning: operator() profile_node %2083 : int[] = prim::profile_ivalue(%2081)
 does not have profile information (Triggered internally at  ../torch/csrc/jit/codegen/cuda/graph_fuser.cpp:104.)
  return forward_call(*input, **kwargs)
/home/ubuntu/OCR_research/env_ocr_research/lib/python3.7/site-packages/torch/nn/modules/module.py:1130: UserWarning: concrete shape for linear input & weight are required to decompose into matmul + bias (Triggered internally at  ../torch/csrc/jit/codegen/cuda/graph_fuser.cpp:2076.)
  return forward_call(*input, **kwargs)


['464', '597', '3.34', '3.77'] ['464', '597', '3.34', '3.77']
torch.Size([4, 3, 32, 128])
['597', "'13N", '498', '647'] ['597', "'13N", '498', '647']
torch.Size([4, 3, 32, 128])
['314', '4#', '224', '348'] ['314', '4#', '224', '348']
torch.Size([4, 3, 32, 128])
['464', '427', '698', '464'] ['464', '427', '698', '464']
torch.Size([4, 3, 32, 128])
['458', '558', '1087', '464'] ['458', '558', '1087', '464']
torch.Size([4, 3, 32, 128])
['297', '472', '298', '98'] ['297', '472', '298', '98']
torch.Size([4, 3, 32, 128])
['674', '387', '422', '467'] ['674', '387', '422', '467']
torch.Size([4, 3, 32, 128])
['538', '538', '547', '397'] ['538', '538', '547', '397']
torch.Size([4, 3, 32, 128])
['474', '497', '997', '497'] ['474', '497', '997', '497']
torch.Size([4, 3, 32, 128])
['224', '417', '417', '538'] ['224', '417', '417', '538']
torch.Size([4, 3, 32, 128])
['497', '494', '497', '$496'] ['497', '494', '497', '$496']
torch.Size([4, 3, 32, 128])
['1897', '5.00', '$224', '594'] ['1897', '5.00',

In [22]:
import cv2
import torch
import os
import time


# reading configuration
charset = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"
BOS = '[B]'
EOS = '[E]'
PAD = '[P]'
specials_first = (EOS,)
specials_last = (BOS, PAD)
itos = specials_first + tuple(charset) + specials_last
stoi = {s: i for i, s in enumerate(itos)}
eos_id, bos_id, pad_id = [stoi[s] for s in specials_first + specials_last]
itos = specials_first + tuple(charset) + specials_last

# decode the model output
def tokenizer_filter(probs, ids):
    ids = ids.tolist()
    try:
        eos_idx = ids.index(eos_id)
    except ValueError:
        eos_idx = len(ids)  # Nothing to truncate.
    # Truncate after EOS
    ids = ids[:eos_idx]
    probs = probs[:eos_idx + 1]  # but include prob. for EOS (if it exists)
    return probs, ids

def ids2tok(token_ids):
    tokens = [itos[i] for i in token_ids]
    return ''.join(tokens)

def decode(token_dists):
    """Decode a batch of token distributions.
    Args:
        token_dists: softmax probabilities over the token distribution. Shape: N, L, C
        raw: return unprocessed labels (will return list of list of strings)

    Returns:
        list of string labels (arbitrary length) and
        their corresponding sequence probabilities as a list of Tensors
    """
    batch_tokens = []
    batch_probs = []
    for dist in token_dists:
        probs, ids = dist.max(-1)  # greedy selection
        probs, ids = tokenizer_filter(probs, ids)
        tokens = ids2tok(ids)
        batch_tokens.append(tokens)
        batch_probs.append(probs)
    return batch_tokens, batch_probs


def img_preprocess(img_orig, device=torch.device("cpu"), nH=32, nW=128):
    # img_orig is a numpy array from cv2.imread(img_path)
    original_image = img_orig[:, :, ::-1]

    resized_image = cv2.resize(original_image, [nW, nH], interpolation=cv2.INTER_CUBIC)
    resized_image = (resized_image - 0.5 * 255) / (0.5 * 255)

    image = torch.as_tensor(resized_image.astype("float32").transpose(2, 0, 1)).unsqueeze(0).to(device)

    return image

def batch_inference(images, model):

    # pre process those images to resize them to the tensor input
    input_tensors = torch.zeros([len(images), 3, 32, 128])

    for i, image in enumerate(images):
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_tensor = img_preprocess(image)

        input_tensors[i, :,:,:] = img_tensor

    # input_tensors = torch.tensor(input_tensors)

    with torch.no_grad():  
        start = time.time()
        logits = model(input_tensors)
        end = time.time()

    print("parseq")
    print(end-start)

    pred = logits.softmax(-1)
    label, confidence = decode(pred)
        
    return label



In [23]:
img_paths = [os.path.join(img_folder, x) for x in os.listdir(img_folder) if x.endswith("jpg")]

In [24]:
images = [cv2.imread(x) for x in img_paths[:5]]
ts_model = torch.jit.load("parseq_cpu_b4_torchscript.pt")


labels = batch_inference(images, ts_model)

print(labels)

parseq
1.1553549766540527
['448', '398', '464', '422', '464']
